In [1]:
import torch
from torch.utils.data import DataLoader, Dataset

import pandas as pd
import numpy as np

import sys
from quantnote import DataLoader, BacktestEngine

In [2]:
engine = BacktestEngine('./MainDB_500.db')

Loading DB...
DB loaded in 50.33 seconds
[initialize] is executed in 58.66 seconds


#### Preprocess macroeconomic data from FRED

In [3]:
def rolling_norm(df, window):
    eps = 0.01
    df = df.dropna()
    return (df-df.rolling(window).mean())/(eps+df.rolling(window).std())

df = None
for key in engine.cache['macro'].keys():
    df_add = engine.cache['macro'][key][['value']]
    df_add.columns = [key]
    try:
        df = pd.merge(df, df_add, left_index=True, right_index=True, how='outer')
    except:
        assert df is None
        df = df_add
        
for ticker in ['^GSPC','^IXIC','^DJI', '^RUT', '^VIX', '^TNX', 'GC=F', 'CL=F']:
    df_add = engine.cache['market'][ticker][['close']]
    df_add.columns = [ticker]
    df = pd.merge(df, df_add, left_index=True, right_index=True, how='outer')
    
df_macro = df.resample('M').mean().ffill()#.dropna()
df_macro = rolling_norm(df_macro, 60)#.dropna()
df_macro.index = df_macro.index.rename('date')

df_macro

,BAAFF,CPIAUCSL,GDPC1,ICSA,IEAMGSN,IEAXGS,INDPRO,ISRATIO,M1SL,PALLFNFINDEXQ,...,UNRATE,WM2NS,^GSPC,^IXIC,^DJI,^RUT,^VIX,^TNX,GC=F,CL=F
date,,,,,,,,,,,,,,,,,,,,,
2000-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31,0.654323,2.085463,1.118899,-0.056393,0.670563,-0.620779,0.109412,-2.048468,4.268433,2.148788,...,0.452090,2.310975,2.600317,2.272397,2.419474,2.676334,0.224534,-0.488283,1.736758,1.147702
2021-06-30,0.425252,2.218695,1.085746,-0.143846,0.425490,-0.025483,-1.582383,-1.668009,3.732504,2.104680,...,0.313289,2.210893,2.552643,2.340625,2.320518,2.768573,-0.095622,-0.598849,1.625949,1.624005
2021-07-31,0.071788,2.414706,1.321496,-0.162450,0.398802,-0.040817,-1.475556,-1.460387,3.314486,3.080205,...,0.349198,2.144792,2.602787,2.424294,2.306143,2.349860,-0.030664,-0.886870,1.420387,1.687762


#### Preprocess coorporate fundamentals from QUANDL

In [4]:
total_tickers = list(set(engine.cache['universe'].ticker))
print(len(total_tickers))


df = None
idx = 0
for ticker in total_tickers:
    print('Progressing {:.2f}%'.format(idx/len(2*total_tickers)*100), end='\r')
    idx += 1
    df_add = engine.cache['metric'][ticker][['ev','evebit','evebitda','marketcap','pb','pe','ps']].resample('M').mean()
    df_add[['ev','marketcap']] = df_add[['ev','marketcap']].apply(np.log)
    df = pd.concat([df, df_add], axis=1)
    
mean_metric = df.T.groupby(level=0).mean().T
std_metric = df.T.groupby(level=0).std().T

df_feature = None
for ticker in total_tickers:
    print('Progressing {:.2f}%'.format(idx/len(2*total_tickers)*100), end='\r')
    idx += 1
    try:
        cols = ['revenue','rnd','opex','netinc','eps',
                'ncf','assets','investments','debt',
                'equity','ebitda','dps','roe','roa',
                'gp','assetturnover','divyield','currentratio']
    
        msg = 'A'
        df_1 = engine.cache['fundamentals'][ticker][cols].resample('M').mean().ffill()
        df_1 = rolling_norm(df_1, 36)
        
        msg = 'B'
        df_2 = engine.cache['market'][ticker][['close','volume']].resample('M')
        df_ohlc = df_2.ohlc()
        df_2 = pd.concat([df_2.mean(), df_2.std()], axis=1)
        df_2.columns = ['cm','vm','cs','vs']
        df_2 = rolling_norm(df_2, 36).ffill()
        df_2['label_feature'] = 1.0*(df_ohlc['close']['close']>df_ohlc['close']['open'])

        msg = 'C'
        df_3 = engine.cache['metric'][ticker][['ev','evebit','evebitda','marketcap','pb','pe','ps']].resample('M').mean()
        df_3[['ev','marketcap']] = df_3[['ev','marketcap']].apply(np.log)

        df_3 = df_3.sub(mean_metric).div(std_metric)#.dropna()

        df = pd.concat([df_1, df_2, df_3], axis=1).replace([np.inf, -np.inf], np.nan).ffill()

        df = df.join(df_macro).ffill().dropna()
        df['ticker'] = ticker

        df_feature = pd.concat([df_feature, df])

    except:
        print('{} is passed for {}'.format(ticker, msg))
    
df_feature.index = df_feature.index.rename('date')
df_feature = df_feature.reset_index().sort_values(['date','ticker'])
col = df_feature.pop('ticker')
df_feature.insert(1,'ticker',col)
df_feature.index =df_feature.date.rename('index')

df_feature

1584
TIIAY is passed for A


,date,ticker,revenue,rnd,opex,netinc,eps,ncf,assets,investments,...,UNRATE,WM2NS,^GSPC,^IXIC,^DJI,^RUT,^VIX,^TNX,GC=F,CL=F
index,,,,,,,,,,,,,,,,,,,,,
2005-07-31,2005-07-31,A,1.234247,-0.966483,-0.946513,1.316936,1.306205,0.924644,0.142073,-0.243097,...,-0.477190,1.452649,0.728764,0.344430,0.786050,2.047629,-1.549043,-0.602274,1.319372,2.605975
2005-07-31,2005-07-31,AAMRQ,2.188809,0.000000,-0.826910,0.779711,0.782966,-1.023643,-0.479839,1.129993,...,-0.477190,1.452649,0.728764,0.344430,0.786050,2.047629,-1.549043,-0.602274,1.319372,2.605975
2005-07-31,2005-07-31,AAPL,2.326395,1.216866,2.067809,2.510725,1.232043,-0.975848,2.336698,2.182357,...,-0.477190,1.452649,0.728764,0.344430,0.786050,2.047629,-1.549043,-0.602274,1.319372,2.605975
2005-07-31,2005-07-31,ABC,0.972645,0.000000,-0.141601,-0.476530,-0.617104,1.025074,-0.282168,0.000000,...,-0.477190,1.452649,0.728764,0.344430,0.786050,2.047629,-1.549043,-0.602274,1.319372,2.605975
2005-07-31,2005-07-31,ABGX,-0.598686,1.281210,-0.819940,0.093534,0.323583,0.593705,0.063780,1.142187,...,-0.477190,1.452649,0.728764,0.344430,0.786050,2.047629,-1.549043,-0.602274,1.319372,2.605975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30,2021-09-30,ZBH,0.653492,1.953338,-1.281332,0.816046,0.790676,0.386058,-0.424803,0.000000,...,0.039121,1.975117,2.511759,2.368177,2.169088,2.216423,-0.020683,-0.846309,1.312171,1.280533
2021-09-30,2021-09-30,ZBRA,3.469203,3.542512,3.507342,2.500823,2.521269,2.644189,2.064884,0.000000,...,0.039121,1.975117,2.511759,2.368177,2.169088,2.216423,-0.020683,-0.846309,1.312171,1.280533
2021-09-30,2021-09-30,ZG,1.500314,1.194490,1.033052,2.407067,2.366708,1.926475,2.014550,-0.298198,...,0.039121,1.975117,2.511759,2.368177,2.169088,2.216423,-0.020683,-0.846309,1.312171,1.280533


#### Generate label

In [5]:
df_label = None
df_market = engine.cache['market']['^SP500TR']['close'].resample('M').ohlc().shift(-1)
df_market = df_market.close.div(df_market.open).apply(np.log)
for ticker in total_tickers:
    try:
        df = engine.cache['market'][ticker]['close'].resample('M').ohlc().shift(-1)
        df = df.close.div(df.open).apply(np.log)
        df = df.sub(df_market).dropna().apply(lambda x:1.*(x>0)).to_frame()
        df.columns = ['label']
        df['ticker'] = ticker
        df_label = pd.concat([df_label, df])
    except:
        print('{} is passed'.format(ticker))

df_label = df_label.reset_index().sort_values(['date','ticker'])
col = df_label.pop('ticker')
df_label.insert(1,'ticker',col)
df_label.index = df_label.date.rename('index')

print(df_label.label.mean())

df_label

0.5041763197514144


,date,ticker,label
index,,,
1997-12-31,1997-12-31,AAMRQ,0.0
1997-12-31,1997-12-31,AAPL,1.0
1997-12-31,1997-12-31,AB,1.0
1997-12-31,1997-12-31,ABC,0.0
1997-12-31,1997-12-31,ABEO,0.0
...,...,...,...
2021-08-31,2021-08-31,ZION,1.0
2021-08-31,2021-08-31,ZM,1.0
2021-08-31,2021-08-31,ZS,0.0


#### Merge label & features in a single df

In [6]:
df_merge = df_label.merge(df_feature, on = ['date','ticker']).sort_values(['ticker','date']).reset_index(drop=True)
df_merge.to_parquet('df_merge.parquet')
df_merge

,date,ticker,label,revenue,rnd,opex,netinc,eps,ncf,assets,...,UNRATE,WM2NS,^GSPC,^IXIC,^DJI,^RUT,^VIX,^TNX,GC=F,CL=F
0,2005-07-31,A,1.0,1.234247,-0.966483,-0.946513,1.316936,1.306205,0.924644,0.142073,...,-0.477190,1.452649,0.728764,0.344430,0.786050,2.047629,-1.549043,-0.602274,1.319372,2.605975
1,2005-08-31,A,1.0,1.171098,-0.921868,-0.897837,1.256818,1.246470,0.986459,0.182608,...,-0.517126,1.444251,0.811326,0.480119,0.813187,1.901277,-1.242244,-0.437151,1.488307,2.922731
2,2005-09-30,A,0.0,0.737334,-0.792459,-0.877424,1.204624,1.190882,1.016461,0.351953,...,-0.704861,1.486153,0.895766,0.608444,0.803532,1.864592,-1.255492,-0.521481,1.730112,2.730995
3,2005-10-31,A,1.0,0.693880,-0.749088,-0.832405,1.152017,1.139030,1.044425,0.415604,...,-0.448388,1.497871,0.693853,0.583807,0.560294,1.481859,-0.895100,0.000479,1.849732,2.286050
4,2005-11-30,A,0.0,0.651364,-0.707375,-0.789754,1.102747,1.090429,1.073711,0.489192,...,-0.669166,1.514691,1.089866,1.050450,0.998649,1.724087,-1.249262,0.194131,1.855772,1.861369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141166,2021-04-30,ZTS,1.0,1.628149,0.943443,1.609527,1.046886,1.025122,1.374082,1.468995,...,0.417270,2.433607,2.721081,2.565117,2.422673,2.978292,-0.049111,-0.488072,1.424093,0.850329
141167,2021-05-31,ZTS,1.0,2.318309,1.356973,1.825324,1.610124,1.554317,1.297361,1.505037,...,0.452090,2.310975,2.600317,2.272397,2.419474,2.676334,0.224534,-0.488283,1.736758,1.147702
141168,2021-06-30,ZTS,1.0,2.147856,1.330654,1.726061,1.561069,1.504829,1.249308,1.435149,...,0.313289,2.210893,2.552643,2.340625,2.320518,2.768573,-0.095622,-0.598849,1.625949,1.624005
141169,2021-07-31,ZTS,0.0,2.011191,1.324566,1.643313,1.530627,1.472170,1.204177,1.375544,...,0.349198,2.144792,2.602787,2.424294,2.306143,2.349860,-0.030664,-0.886870,1.420387,1.687762


### Generate torch dataset

In [7]:
class MyDataset(Dataset):
    def __init__(self, feature_arr, label_arr):
        self.feature_arr = feature_arr
        self.label_arr = label_arr
        assert len(feature_arr) == len(label_arr)
        
    def __len__(self):
        return len(self.label_arr)

    def __getitem__(self, idx):
        feature = self.feature_arr[idx]
        label = self.label_arr[idx]
        return feature, label
    
def layer_norm(arr):
    for idx in range(arr.shape[1]):
        arr_1d = arr[:,idx]
        mean = np.mean(arr_1d)
        std = np.std(arr_1d)
        arr[:,idx] = (arr_1d-mean)/(0.001+std)
    return arr

In [8]:
def generate_dataset(train_date):
    WINDOW = 60

    feature_arr_train = []
    label_arr_train = []
    feature_arr_val = []
    label_arr_val = []
    
    for idx in df_merge.index:
        print('Progressing {:.2f}%'.format(idx/len(df_merge)*100), end='\r')
        try:
            snapshot = df_merge.loc[:idx]
            msg='A'
            assert len(snapshot) >= WINDOW
            snapshot = df_merge.loc[:idx].iloc[-WINDOW:].reset_index(drop=True)
            position = snapshot.index/len(snapshot.index)
            snapshot.insert(len(snapshot.columns),'position',position)

            tickers = list(set(snapshot.ticker))
            msg='B'
            assert len(tickers) == 1

            date = snapshot.iloc[-1].date
            label = np.array([snapshot.iloc[-1].label])
            feature = snapshot[snapshot.columns[3:]].values
            msg='C'
            assert len(feature) == WINDOW

            feature = layer_norm(feature)

            token = np.random.rand()

            if pd.to_datetime(date) < pd.to_datetime(train_date):
                if token < 0.8:
                    label_arr_train.append(label)
                    feature_arr_train.append(feature)
                else:
                    label_arr_val.append(label)
                    feature_arr_val.append(feature)
        except:
            # print(msg)
            pass

    feature_arr_train = np.array(feature_arr_train)
    label_arr_train = np.array(label_arr_train)
    feature_arr_val = np.array(feature_arr_val)
    label_arr_val = np.array(label_arr_val)
    
    print(feature_arr_train.shape, feature_arr_val.shape)
    print(label_arr_train.shape, label_arr_val.shape)
    
    train_dataset = MyDataset(feature_arr_train, label_arr_train)
    val_dataset = MyDataset(feature_arr_val, label_arr_val)

    torch.save(train_dataset, 'train_datatset_{}.pt'.format(train_date))
    torch.save(val_dataset, 'val_datatset_{}.pt'.format(train_date))
               
    print('{} is saved'.format(train_date))

In [9]:
generate_dataset('2018-01-01')
generate_dataset('2019-01-01')
generate_dataset('2020-01-01')
generate_dataset('2021-01-01')

(46339, 60, 54) (11332, 60, 54)
(46339, 1) (11332, 1)
2018-01-01 is saved
(52033, 60, 54) (13201, 60, 54)
(52033, 1) (13201, 1)
2019-01-01 is saved
(58161, 60, 54) (14540, 60, 54)
(58161, 1) (14540, 1)
2020-01-01 is saved
(64193, 60, 54) (15988, 60, 54)
(64193, 1) (15988, 1)
2021-01-01 is saved
